### Analyzing public and hybrid cloud networks

XXXX

In [33]:
# Import packages and load questions
%run startup.py
load_questions()

# a helper function to print the flow and the first trace 
def show_first_trace(traceroute_answer_frame):
    show("Flow: {}".format(traceroute_answer_frame.iloc[0]['Flow']))
    show(traceroute_answer_frame.iloc[0]['Traces'][0])
    
def is_reachable(start_location, end_location, headers=None):
    ans = bfq.reachability(pathConstraints=PathConstraints(startLocation=start_location, 
                                                           endLocation=end_location),
                          headers=headers,
                          actions="success").answer()
    show(ans.frame())
    return len(ans.frame()) > 0

### Initializing the Network and Snapshot

SNAPSHOT_PATH below can be updated to point to a custom snapshot directory, see the Batfish instructions for how to package data for analysis. More example networks are available in the networks folder of the Batfish repository.

In [24]:
# Initialize a network and snapshot
NETWORK_NAME = "hybrid-cloud"
SNAPSHOT_NAME = "snapshot"

SNAPSHOT_PATH = "networks/hybrid-cloud"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True, extra_args={ "parsereuse" : False})

'snapshot'

The network snapshot that we initialized above is illustrated below. It has a physical datacenter network on the left, which has a standard leaf spine design, along with an example host. The exit gateway of the datacenter connects to an ISP called "orange".

The AWS network is shown on the right. It is spread across two regions. Each region has two VPCs, one of which is meant to host Internet-facing services and the other is meant to host only private services. The two VPCs in a region peer via a transit gateway. Each VPC has two subnets, and we have some instances running as well. 

The physical network connects to the AWS network by having the exitgw establish IPSec tunnels, shown in pink, to the two transit gateways. BGP sessions running atop these tunnels to make endpoints aware of networks on the other side.

You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/hybrid-cloud). The AWS portion of the configuration is in the aws_configs subfolder. It has JSON files obtained via AWS APIs. An example script that packages AWS data into a Batfish snapshot is [here](https://github.com/ratulm/bf-aws-snapshot).

![example-bgp-network](https://raw.githubusercontent.com/batfish/pybatfish/aws-notebook/jupyter_notebooks/networks/hybrid-cloud/hybrid-cloud.png)

In [3]:
host_dc = "srv-101"

## instances in AWS in each region and VPC type (public, private)
host_east2_private = "i-04cd3db5124a05ee6"
host_east2_public = "i-01602d9efaed4409a"
host_west2_private = "i-0a5d64b8b58c6dd09"
host_west2_public = "i-02cae6eaa9edeed70"

## public IPs of instances in AWS
ip_east2_public = "13.59.144.125" # of i-01602d9efaed4409a
ip_west2_public = "54.191.42.182" # of i-02cae6eaa9edeed70

#### Paths across VPCs within an AWS region

In [ ]:
# traceroute between instances in the same region
ans = bfq.traceroute(startLocation=host_east2_public, 
                     headers=HeaderConstraints(dstIps=host_east2_private, 
                                               applications="http")).answer()
show_first_trace(ans.frame())

XXXX explain transit gateway, network acl, security groups

#### Paths across AWS regions

In [8]:
# traceroute betwee instances across region using the destination's private IP
ans = bfq.traceroute(startLocation=host_east2_public, 
                     headers=HeaderConstraints(dstIps=host_west2_public,
                                              applications="ssh")).answer()
show_first_trace(ans.frame())

'Flow: start=i-01602d9efaed4409a [10.20.1.207:49152->10.40.2.80:22 TCP length=512]'

Trace(disposition='NULL_ROUTED', hops=[Hop(node='i-01602d9efaed4409a', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '10.20.1.1', 'protocol': 'static'}], arpIp='10.20.1.1', outputInterface='eni-01997085076a9b98a'), action='FORWARDED'), Step(detail=FilterStepDetail(filter='~SECURITY_GROUP_EGRESS_ACL~', filterType='EGRESS_FILTER', inputInterface='', flow=Flow(dscp=0, dstIp='10.40.2.80', dstPort=22, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='i-01602d9efaed4409a', ingressVrf='default', ipProtocol='TCP', packetLength=512, srcIp='10.20.1.207', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=1, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=SetupSessionStepDetail(incomingInterfaces=['eni-01997085076a9b98a'], sessionAction=Accept(), matchCriteria=SessionMatchExpr(ipProtocol='TCP', srcIp='10.40.2.80', dstIp='10.20.1.207', srcPort=22, dstPort=49152), transformation=[]), action='SETUP_SESSION'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='eni-01997085076a9b98a', transformedFlow=None), action='TRANSMITTED')]), Hop(node='subnet-06a692ed4ef84368d', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='subnet-06a692ed4ef84368d', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='acl-09c0bb4e71ae5f9e4_egress', filterType='INGRESS_FILTER', inputInterface='subnet-06a692ed4ef84368d', flow=Flow(dscp=0, dstIp='10.40.2.80', dstPort=22, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='i-01602d9efaed4409a', ingressVrf='default', ipProtocol='TCP', packetLength=512, srcIp='10.20.1.207', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=1, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '169.254.0.2', 'protocol': 'static'}], arpIp='169.254.0.2', outputInterface='igw-02fd68f94367a67c7'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='igw-02fd68f94367a67c7', transformedFlow=None), action='TRANSMITTED')]), Hop(node='igw-02fd68f94367a67c7', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='subnet-06a692ed4ef84368d', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '240.0.0.1', 'protocol': 'bgp'}], arpIp='240.0.0.1', outputInterface='backbone'), action='FORWARDED'), Step(detail=TransformationStepDetail(transformationType='SOURCE_NAT', flowDiffs=[FlowDiff(fieldName='srcIp', oldValue='10.20.1.207', newValue='13.59.144.125')]), action='TRANSFORMED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='backbone', transformedFlow={'dscp': 0, 'dstIp': '10.40.2.80', 'dstPort': 22, 'ecn': 0, 'fragmentOffset': 0, 'ingressNode': 'i-01602d9efaed4409a', 'ingressVrf': 'default', 'ipProtocol': 'TCP', 'packetLength': 512, 'srcIp': '13.59.144.125', 'srcPort': 49152, 'state': 'NEW', 'tag': 'tag', 'tcpFlagsAck': 0, 'tcpFlagsCwr': 0, 'tcpFlagsEce': 0, 'tcpFlagsFin': 0, 'tcpFlagsPsh': 0, 'tcpFlagsRst': 0, 'tcpFlagsSyn': 1, 'tcpFlagsUrg': 0}), action='TRANSMITTED')]), Hop(node='aws-backbone', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='~Interface_1~', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '240.1.1.4', 'protocol': 'bgp'}], arpIp='240.1.1.4', outputInterface='~Interface_6~'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='~Interface_6~', transformedFlow={'dscp': 0, 'dstIp': '10.40.2.80', 'dstPort': 22, 'ecn': 0, 'fragmentOffset': 0, 'ingressNode': 'i-01602d9efaed4409a', 'ingressVrf': 'default', 'ipProtocol': 'TCP', 'packetLength': 512, 'srcIp': '13.59.144.125', 'srcPort': 49152, 'state': 'NEW', 'tag': 

In [9]:
# traceroute betwee instances across region using the destination's public IP
ans = bfq.traceroute(startLocation=host_east2_public, 
                     headers=HeaderConstraints(dstIps=ip_west2_public,
                                              applications="ssh")).answer()
show_first_trace(ans.frame())

'Flow: start=i-01602d9efaed4409a [10.20.1.207:49152->54.191.42.182:22 TCP length=512]'

Trace(disposition='ACCEPTED', hops=[Hop(node='i-01602d9efaed4409a', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '10.20.1.1', 'protocol': 'static'}], arpIp='10.20.1.1', outputInterface='eni-01997085076a9b98a'), action='FORWARDED'), Step(detail=FilterStepDetail(filter='~SECURITY_GROUP_EGRESS_ACL~', filterType='EGRESS_FILTER', inputInterface='', flow=Flow(dscp=0, dstIp='54.191.42.182', dstPort=22, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='i-01602d9efaed4409a', ingressVrf='default', ipProtocol='TCP', packetLength=512, srcIp='10.20.1.207', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=1, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=SetupSessionStepDetail(incomingInterfaces=['eni-01997085076a9b98a'], sessionAction=Accept(), matchCriteria=SessionMatchExpr(ipProtocol='TCP', srcIp='54.191.42.182', dstIp='10.20.1.207', srcPort=22, dstPort=49152), transformation=[]), action='SETUP_SESSION'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='eni-01997085076a9b98a', transformedFlow=None), action='TRANSMITTED')]), Hop(node='subnet-06a692ed4ef84368d', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='subnet-06a692ed4ef84368d', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='acl-09c0bb4e71ae5f9e4_egress', filterType='INGRESS_FILTER', inputInterface='subnet-06a692ed4ef84368d', flow=Flow(dscp=0, dstIp='54.191.42.182', dstPort=22, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='i-01602d9efaed4409a', ingressVrf='default', ipProtocol='TCP', packetLength=512, srcIp='10.20.1.207', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=1, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '169.254.0.2', 'protocol': 'static'}], arpIp='169.254.0.2', outputInterface='igw-02fd68f94367a67c7'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='igw-02fd68f94367a67c7', transformedFlow=None), action='TRANSMITTED')]), Hop(node='igw-02fd68f94367a67c7', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='subnet-06a692ed4ef84368d', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '240.0.0.1', 'protocol': 'bgp'}], arpIp='240.0.0.1', outputInterface='backbone'), action='FORWARDED'), Step(detail=TransformationStepDetail(transformationType='SOURCE_NAT', flowDiffs=[FlowDiff(fieldName='srcIp', oldValue='10.20.1.207', newValue='13.59.144.125')]), action='TRANSFORMED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='backbone', transformedFlow={'dscp': 0, 'dstIp': '54.191.42.182', 'dstPort': 22, 'ecn': 0, 'fragmentOffset': 0, 'ingressNode': 'i-01602d9efaed4409a', 'ingressVrf': 'default', 'ipProtocol': 'TCP', 'packetLength': 512, 'srcIp': '13.59.144.125', 'srcPort': 49152, 'state': 'NEW', 'tag': 'tag', 'tcpFlagsAck': 0, 'tcpFlagsCwr': 0, 'tcpFlagsEce': 0, 'tcpFlagsFin': 0, 'tcpFlagsPsh': 0, 'tcpFlagsRst': 0, 'tcpFlagsSyn': 1, 'tcpFlagsUrg': 0}), action='TRANSMITTED')]), Hop(node='aws-backbone', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='~Interface_1~', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '54.191.42.182/32', 'nextHopIp': '240.0.0.2', 'protocol': 'bgp'}], arpIp='240.0.0.2', outputInterface='~Interface_2~'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='~Interface_2~', transformedFlow={'dscp': 0, 'dstIp': '54.191.42.182', 'dstPort': 22, 'ecn': 0, 'fragmentOffset': 0, 'ingressNode': 'i-01602d9efaed4409a', 'ingressVrf': 'default', 'ipProtocol': 'TCP', 'packetLength': 512, 'srcIp': '13.59.144.125', 'srcPort': 49152, 'sta

#### Paths from the physical DC to AWS

In [25]:
# traceroute from DC host to an instances using private IP
ans = bfq.traceroute(startLocation=host_dc, 
                     headers=HeaderConstraints(dstIps=host_east2_public,
                                              applications="http")).answer()
show_first_trace(ans.frame())

'Flow: start=srv-101 [203.0.113.12:49152->10.20.1.207:80 TCP length=512]'

Trace(disposition='ACCEPTED', hops=[Hop(node='srv-101', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '203.0.113.2', 'protocol': 'static'}], arpIp='203.0.113.2', outputInterface='eth0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='eth0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='leaf1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='Ethernet10', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '10.20.0.0/16', 'nextHopIp': '10.10.11.1', 'protocol': 'bgp'}], arpIp='10.10.11.1', outputInterface='Ethernet1'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='Ethernet1', transformedFlow=None), action='TRANSMITTED')]), Hop(node='spine1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='Ethernet1', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '10.20.0.0/16', 'nextHopIp': '10.10.100.2', 'protocol': 'bgp'}], arpIp='10.10.100.2', outputInterface='Ethernet10'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='Ethernet10', transformedFlow=None), action='TRANSMITTED')]), Hop(node='exitgw', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '10.20.0.0/16', 'nextHopIp': '169.254.25.161', 'protocol': 'bgp'}], arpIp='169.254.25.161', outputInterface='Tunnel1'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='Tunnel1', transformedFlow=None), action='TRANSMITTED')]), Hop(node='tgw-06b348adabd13452d', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='vpn-vpn-01c45673532d3e33e-1', inputVrf='vrf-tgw-rtb-00e37bc5142347b03'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '10.20.0.0/16', 'nextHopIp': '169.254.0.2', 'protocol': 'static'}], arpIp='169.254.0.2', outputInterface='vpc-0574d08f8d05917e4-tgw-attach-0648110513acd6de5'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='vpc-0574d08f8d05917e4-tgw-attach-0648110513acd6de5', transformedFlow=None), action='TRANSMITTED')]), Hop(node='vpc-0574d08f8d05917e4', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='tgw-06b348adabd13452d-tgw-attach-0648110513acd6de5', inputVrf='vrf-tgw-attach-0648110513acd6de5'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '10.20.1.0/24', 'nextHopIp': '169.254.0.1', 'protocol': 'static'}], arpIp='169.254.0.1', outputInterface='subnet-06a692ed4ef84368d-tgw-attach-0648110513acd6de5'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='subnet-06a692ed4ef84368d-tgw-attach-0648110513acd6de5', transformedFlow=None), action='TRANSMITTED')]), Hop(node='subnet-06a692ed4ef84368d', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='vpc-0574d08f8d05917e4-tgw-attach-0648110513acd6de5', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '10.20.1.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='subnet-06a692ed4ef84368d'), action='FORWARDED'), Step(detail=FilterStepDetail(filter='acl-09c0bb4e71ae5f9e4_ingress', filterType='EGRESS_FILTER', inputInterface='vpc-0574d08f8d05917e4-tgw-attach-0648110513acd6de5', flow=Flow(dscp=0, dstIp='10.20.1.207', dstPort=80, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='srv-101', ingressVrf='default', ipProtocol='TCP', packetLength=512, srcIp='203.0.113.12', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=1, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='subnet-06a692ed4ef84368d', transformedFlow=None), ac

In [11]:
# traceroute from DC host to an instances using public IP
ans = bfq.traceroute(startLocation=host_dc, 
                     headers=HeaderConstraints(dstIps=ip_east2_public,
                                              applications="http")).answer()
show_first_trace(ans.frame())

'Flow: start=srv-101 [203.0.113.12:49152->13.59.144.125:80 TCP length=512]'

Trace(disposition='DENIED_IN', hops=[Hop(node='srv-101', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '203.0.113.2', 'protocol': 'static'}], arpIp='203.0.113.2', outputInterface='eth0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='eth0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='leaf1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='Ethernet10', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '10.10.11.1', 'protocol': 'bgp'}], arpIp='10.10.11.1', outputInterface='Ethernet1'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='Ethernet1', transformedFlow=None), action='TRANSMITTED')]), Hop(node='spine1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='Ethernet1', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '10.10.100.2', 'protocol': 'bgp'}], arpIp='10.10.100.2', outputInterface='Ethernet10'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='Ethernet10', transformedFlow=None), action='TRANSMITTED')]), Hop(node='exitgw', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '147.75.69.26', 'protocol': 'static'}], arpIp='147.75.69.26', outputInterface='GigabitEthernet3'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet3', transformedFlow=None), action='TRANSMITTED')]), Hop(node='isp_dc', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='~Interface_0~', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '0.0.0.0/0', 'nextHopIp': '240.1.1.2', 'protocol': 'bgp'}], arpIp='240.1.1.2', outputInterface='~Interface_4~'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='~Interface_4~', transformedFlow=None), action='TRANSMITTED')]), Hop(node='internet', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='~Interface_3~', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '13.59.144.125/32', 'nextHopIp': '240.1.1.5', 'protocol': 'bgp'}], arpIp='240.1.1.5', outputInterface='~Interface_5~'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='~Interface_5~', transformedFlow=None), action='TRANSMITTED')]), Hop(node='aws-backbone', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='~Interface_6~', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '13.59.144.125/32', 'nextHopIp': '240.0.0.0', 'protocol': 'bgp'}], arpIp='240.0.0.0', outputInterface='~Interface_1~'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='~Interface_1~', transformedFlow=None), action='TRANSMITTED')]), Hop(node='igw-02fd68f94367a67c7', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='backbone', inputVrf='default'), action='RECEIVED'), Step(detail=TransformationStepDetail(transformationType='DEST_NAT', flowDiffs=[FlowDiff(fieldName='dstIp', oldValue='13.59.144.125', newValue='10.20.1.207')]), action='TRANSFORMED'), Step(detail=RoutingStepDetail(routes=[{'network': '10.20.1.0/24', 'nextHopIp': '169.254.0.1', 'protocol': 'static'}], arpIp='169.254.0.1', outputInterface='subnet-06a692ed4ef84368d'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='subnet-06a692ed4ef84368d', transformedFlow={'dscp': 0, 'dstIp': '10.20.1.207', 'dstPort': 80, 'ecn': 0, 'fragmentOffset': 0, 'ingressNode': 'srv-101', 'ingressVrf': 'default', 'ipProtocol': 'TCP', 'packetLength': 512, 'srcIp': '203.0.113.12', 'srcPort': 49152, 'state': 'NEW', 'tag': 'tag', 'tcpFlagsAck': 0, 

## Which instances are accessible from the Internet?

In [12]:
all_instances = [host_east2_public, host_east2_private, host_west2_public, host_west2_private]

reachable_from_internet = [host for host in all_instances if is_reachable("internet", host)]
isolated_from_internet = [host for host in all_instances if not is_reachable("internet", host)]
reachable_from_dc = [host for host in all_instances if is_reachable(host_dc, host)]

print("Instances reachable from the Internet: {}\n".format(reachable_from_internet))
print("Instances NOT reachable from the Internet: {}\n".format(isolated_from_internet))
print("Instances reachable from the DC: {}\n".format(reachable_from_dc))

Instances reachable from the Internet: ['i-01602d9efaed4409a', 'i-02cae6eaa9edeed70']

Instances NOT reachable from the Internet: ['i-04cd3db5124a05ee6', 'i-0a5d64b8b58c6dd09']

Instances reachable from the DC: ['i-01602d9efaed4409a', 'i-04cd3db5124a05ee6', 'i-02cae6eaa9edeed70', 'i-0a5d64b8b58c6dd09']



In [32]:
reachable_from_internet_non_ssh = [host for host in all_instances if is_reachable("internet", host, HeaderConstraints(applications="!ssh"))]
reachable_from_internet_non_ssh

,Flow,Traces,TraceCount
0,Start Location: internetSrc IP: 240.1.1.2Src Port: 49152Dst IP: 13.59.144.125Dst Port: 23IP Protocol: TCP,"ACCEPTED1. node: internet ORIGINATED(default) FORWARDED(ARP IP: 240.1.1.5, Output Interface: ~Interface_5~, Routes: [bgp (Network: 13.59.144.125/32, Next Hop IP:240.1.1.5)]) TRANSMITTED(~Interface_5~)2. node: aws-backbone RECEIVED(~Interface_6~) FORWARDED(ARP IP: 240.0.0.0, Output Interface: ~Interface_1~, Routes: [bgp (Network: 13.59.144.125/32, Next Hop IP:240.0.0.0)]) TRANSMITTED(~Interface_1~)3. node: igw-02fd68f94367a67c7 RECEIVED(backbone) TRANSFORMED(DEST_NAT dstIp: 13.59.144.125 -> 10.20.1.207) FORWARDED(ARP IP: 169.254.0.1, Output Interface: subnet-06a692ed4ef84368d, Routes: [static (Network: 10.20.1.0/24, Next Hop IP:169.254.0.1)]) TRANSMITTED(subnet-06a692ed4ef84368d)4. node: subnet-06a692ed4ef84368d RECEIVED(igw-02fd68f94367a67c7) FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: subnet-06a692ed4ef84368d, Routes: [connected (Network: 10.20.1.0/24, Next Hop IP:AUTO/NONE(-1l))]) PERMITTED(acl-09c0bb4e71ae5f9e4_ingress (EGRESS_FILTER)) TRANSMITTED(subnet-06a692ed4ef84368d)5. node: i-01602d9efaed4409a RECEIVED(eni-01997085076a9b98a) PERMITTED(Security Group launch-wizard-1 (INGRESS_FILTER)) ACCEPTED(eni-01997085076a9b98a)",1


,Flow,Traces,TraceCount


,Flow,Traces,TraceCount
0,Start Location: internetSrc IP: 240.1.1.2Src Port: 49152Dst IP: 54.191.42.182Dst Port: 23IP Protocol: TCP,"DENIED_IN1. node: internet ORIGINATED(default) FORWARDED(ARP IP: 240.1.1.5, Output Interface: ~Interface_5~, Routes: [bgp (Network: 54.191.42.182/32, Next Hop IP:240.1.1.5)]) TRANSMITTED(~Interface_5~)2. node: aws-backbone RECEIVED(~Interface_6~) FORWARDED(ARP IP: 240.0.0.2, Output Interface: ~Interface_2~, Routes: [bgp (Network: 54.191.42.182/32, Next Hop IP:240.0.0.2)]) TRANSMITTED(~Interface_2~)3. node: igw-0a8309f3192e7cea3 RECEIVED(backbone) TRANSFORMED(DEST_NAT dstIp: 54.191.42.182 -> 10.40.2.80) FORWARDED(ARP IP: 169.254.0.1, Output Interface: subnet-06005943afe32f714, Routes: [static (Network: 10.40.2.0/24, Next Hop IP:169.254.0.1)]) TRANSMITTED(subnet-06005943afe32f714)4. node: subnet-06005943afe32f714 RECEIVED(igw-0a8309f3192e7cea3) FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: subnet-06005943afe32f714, Routes: [connected (Network: 10.40.2.0/24, Next Hop IP:AUTO/NONE(-1l))]) PERMITTED(acl-087574e8620270842_ingress (EGRESS_FILTER)) TRANSMITTED(subnet-06005943afe32f714)5. node: i-02cae6eaa9edeed70 RECEIVED(eni-087e18628dadd9b48) DENIED(Security Group launch-wizard-1 (INGRESS_FILTER))",1


,Flow,Traces,TraceCount


['i-01602d9efaed4409a', 'i-02cae6eaa9edeed70']

In [38]:
ans = bfq.reachability(pathConstraints=PathConstraints(startLocation="internet"), 
                       headers=HeaderConstraints(dstIps="54.191.42.182"),
                       actions="success").answer()
show(ans)

,Flow,Traces,TraceCount
0,Start Location: internetSrc IP: 240.1.1.2Dst IP: 54.191.42.182IP Protocol: ICMP,"DENIED_IN1. node: internet ORIGINATED(default) FORWARDED(ARP IP: 240.1.1.5, Output Interface: ~Interface_5~, Routes: [bgp (Network: 54.191.42.182/32, Next Hop IP:240.1.1.5)]) TRANSMITTED(~Interface_5~)2. node: aws-backbone RECEIVED(~Interface_6~) FORWARDED(ARP IP: 240.0.0.2, Output Interface: ~Interface_2~, Routes: [bgp (Network: 54.191.42.182/32, Next Hop IP:240.0.0.2)]) TRANSMITTED(~Interface_2~)3. node: igw-0a8309f3192e7cea3 RECEIVED(backbone) TRANSFORMED(DEST_NAT dstIp: 54.191.42.182 -> 10.40.2.80) FORWARDED(ARP IP: 169.254.0.1, Output Interface: subnet-06005943afe32f714, Routes: [static (Network: 10.40.2.0/24, Next Hop IP:169.254.0.1)]) TRANSMITTED(subnet-06005943afe32f714)4. node: subnet-06005943afe32f714 RECEIVED(igw-0a8309f3192e7cea3) FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: subnet-06005943afe32f714, Routes: [connected (Network: 10.40.2.0/24, Next Hop IP:AUTO/NONE(-1l))]) PERMITTED(acl-087574e8620270842_ingress (EGRESS_FILTER)) TRANSMITTED(subnet-06005943afe32f714)5. node: i-02cae6eaa9edeed70 RECEIVED(eni-087e18628dadd9b48) DENIED(Security Group launch-wizard-1 (INGRESS_FILTER))",1


In [27]:
bfq.initIssues().answer()

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,"['leaf1', 'leaf2', 'leaf3', 'leaf4', 'spine1', 'spine2']",None,Convert warning (redflag),Redistribution of OSPF_NSSA_EXTERNAL routes is not yet supported,None,None
1,"['leaf1', 'leaf2', 'leaf3', 'leaf4', 'spine1', 'spine2']",None,Convert warning (redflag),Redistribution of OSPF_NSSA_EXTERNAL_TYPE_1 routes is not yet supported,None,None
2,"['leaf1', 'leaf2', 'leaf3', 'leaf4', 'spine1', 'spine2']",None,Convert warning (redflag),Redistribution of OSPF_NSSA_EXTERNAL_TYPE_2 routes is not yet supported,None,None
3,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-east-2/VpcEndpointServices.json,None,None
4,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-west-2/VpcEndpointServices.json,None,None
